In [ ]:
!pip install opencv-python imutils pynput

In [ ]:
import cv2
import imutils
import numpy as np
from pynput import mouse

In [ ]:
# Constants
THRESHOLD = 95
ROI_SIZE = 300
MIN_ASPECT_RATIO = 0.2
MAX_ASPECT_RATIO = 1.8

In [ ]:
# Set up webcam and mouse controller
cap = cv2.VideoCapture(0)
mouse_controller = mouse.Controller()

In [ ]:
def detect_hand_gesture(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Blur the frame to reduce noise
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the frame to create a binary image
    _, thresh = cv2.threshold(gray, THRESHOLD, 255, cv2.THRESH_BINARY)

    # Find contours in the frame
    cnts, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Check if a contour was found
    if len(cnts) > 0:
        # Find the largest contour
        c = max(cnts, key=cv2.contourArea)

        # Compute the bounding box for the contour
        x, y, w, h = cv2.boundingRect(c)

        # Compute the aspect ratio of the bounding box
        aspect_ratio = w / float(h)

        # Check if the aspect ratio is within the desired range
        if MIN_ASPECT_RATIO < aspect_ratio < MAX_ASPECT_RATIO:
            # Return True to indicate that a hand gesture was detected
            return True

    # Return False if no hand gesture was detected
    return False


In [ ]:
def move_mouse(frame, roi):
    # Find the center of the ROI
    roii = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    M = cv2.moments(roii)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    # Calculate the movement of the mouse based on the ROI's center
    dx = cx - ROI_SIZE // 2
    dy = cy - ROI_SIZE // 2

    # Move the mouse
    mouse_controller.move(dx, dy)


In [ ]:
while True:
    # Capture a frame from the webcam
    _, frame = cap.read()

    # Crop the frame to a ROI centered on the bottom half of the frame
    h, w = frame.shape[:2]
    roi = frame[h//2:, w//2-ROI_SIZE//2:w//2+ROI_SIZE//2]

    # Check if a hand gesture was detected in the ROI
    if detect_hand_gesture(roi):
        # Move the mouse based on the position of the hand in the ROI
        move_mouse(frame, roi)

    # Overlay the ROI and the mouse cursor on the frame
    frame = cv2.circle(frame, mouse_controller.position, 5, (0, 0, 255), -1)
    cv2.rectangle(frame, (w//2-ROI_SIZE//2, h//2), (w//2+ROI_SIZE//2, h), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow("Frame", frame)


    # Check if the user pressed the 'q' key to quit
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
